In [ ]:
from google.colab import drive
import os
from transformers import pipeline
from PIL import Image
import numpy as np
from tqdm import tqdm

# Google Drive를 마운트합니다.
drive.mount('/content/drive')

# 깊이 추정 파이프라인 로드
checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
pipe = pipeline("depth-estimation", model=checkpoint)

def process_image_folder(folder_path, rgb_output_folder, depth_output_folder, rgbd_output_folder):
    if not os.path.exists(rgb_output_folder):
        os.makedirs(rgb_output_folder)
    if not os.path.exists(depth_output_folder):
        os.makedirs(depth_output_folder)
    if not os.path.exists(rgbd_output_folder):
        os.makedirs(rgbd_output_folder)

    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

    for idx, image_file in enumerate(tqdm(image_files, desc=f"Processing {folder_path.split('/')[-1]}")):
        image_path = os.path.join(folder_path, image_file)
        image = Image.open(image_path)

        # 파일 이름 생성
        class_name = folder_path.split('/')[-1]  # 클래스 이름
        image_index = idx + 1                    # 1부터 시작하는 인덱스

        # RGB 이미지를 numpy 배열로 변환
        rgb_image = np.array(image)

        # RGB 이미지를 0-255 범위로 정규화 (필요한 경우)
        if rgb_image.max() > 255:  # 정규화가 필요한 경우
            rgb_image_normalized = (rgb_image * 255 / rgb_image.max()).astype('uint8')
        else:
            rgb_image_normalized = rgb_image

        # RGB 이미지를 PIL 이미지로 변환
        rgb_image_pil = Image.fromarray(rgb_image_normalized)

        # RGB 이미지를 저장
        rgb_image_filename = f"rgb_{class_name}_{image_index}.jpg"
        rgb_image_path = os.path.join(rgb_output_folder, rgb_image_filename)
        rgb_image_pil.save(rgb_image_path)

        # 깊이 추정 예측
        predictions = pipe(image)

        # 깊이 맵을 numpy 배열로 변환
        depth_map = predictions['predicted_depth'].squeeze().cpu().numpy()

        # 깊이 맵을 0-255 범위로 정규화하여 흑백 이미지로 저장
        normalized_depth_map = (depth_map * 255 / depth_map.max()).astype('uint8')
        depth_map_image = Image.fromarray(normalized_depth_map)

        # 깊이 맵 크기를 RGB 이미지 크기에 맞게 조정
        depth_map_resized = depth_map_image.resize(image.size, Image.ANTIALIAS)
        depth_map_filename = f"depth_{class_name}_{image_index}.png"
        depth_map_path = os.path.join(depth_output_folder, depth_map_filename)
        depth_map_resized.save(depth_map_path)

        # RGB 이미지 크기를 깊이 맵 크기에 맞게 조정
        rgb_image_resized = rgb_image_pil.resize(depth_map_resized.size, Image.ANTIALIAS)

        # 깊이 맵 크기와 동일한 RGB 이미지와 깊이 맵을 결합하여 RGBD 이미지 생성
        rgbd_image = np.dstack((np.array(rgb_image_resized), np.array(depth_map_resized)))

        # RGBD 이미지를 저장
        rgbd_image_filename = f"rgbd_{class_name}_{image_index}.png"
        rgbd_image_path = os.path.join(rgbd_output_folder, rgbd_image_filename)
        rgbd_image_pil = Image.fromarray(rgbd_image)
        rgbd_image_pil.save(rgbd_image_path)

# 각 클래스 폴더에 대해 깊이 추정을 수행하고 결과를 저장합니다.
voc_top5_path = '/content/drive/My Drive/data/voc_top5'
for class_folder in os.listdir(voc_top5_path):
    class_path = os.path.join(voc_top5_path, class_folder)

    rgb_output_folder = os.path.join('/content/drive/My Drive/data/voc_top5/rgb_images', class_folder)
    depth_output_folder = os.path.join('/content/drive/My Drive/data/voc_top5/depth_maps', class_folder)
    rgbd_output_folder = os.path.join('/content/drive/My Drive/data/voc_top5/rgbd_images', class_folder)

    process_image_folder(class_path, rgb_output_folder, depth_output_folder, rgbd_output_folder)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Processing horse:   0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-1-47dc7f8449da>:61: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  depth_map_resized = depth_map_image.resize(image.size, Image.ANTIALIAS)
<ipython-input-1-47dc7f8449da>:67: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  rgb_image_re

In [ ]:
from google.colab import drive
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Google Drive를 마운트합니다.
drive.mount('/content/drive')


def visualize_and_load_images(base_folder_path, img_type='depth'):
    class_folders = [f for f in os.listdir(base_folder_path) if os.path.isdir(os.path.join(base_folder_path, f))]
    images = {}

    plt.figure(figsize=(20, 25))

    for idx, class_folder in enumerate(class_folders):
        folder_path = os.path.join(base_folder_path, class_folder)
        files = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]

        if files:
            file = files[0]  # 각 클래스의 첫 번째 파일만 사용
            file_path = os.path.join(folder_path, file)
            image = Image.open(file_path)

            # 이미지를 numpy 배열로 변환
            image_array = np.array(image)
            images[class_folder] = image_array

            # 이미지 시각화
            plt.subplot(5, 4, idx + 1)
            plt.imshow(image_array, cmap='gray' if img_type == 'depth' else None)
            plt.title(f'{class_folder} - {file}')
            plt.axis('off')
            plt.colorbar()

    plt.suptitle(f'Class-wise {img_type.capitalize()} Maps', fontsize=30)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    return images

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 전체 voc_top5 폴더에 대해 깊이 맵을 시각화하고 로드합니다.
base_depth_folder = '/content/drive/My Drive/data/voc_top5/depth_maps'
depth_maps = visualize_and_load_images(base_depth_folder, img_type='depth')

# 전체 voc_top5 폴더에 대해 RGB 이미지를 시각화하고 로드합니다.
base_rgb_folder = '/content/drive/My Drive/data/voc_top5/rgb_images'
rgb_images = visualize_and_load_images(base_rgb_folder, img_type='rgb')

# 예시로 하나의 클래스에서 첫 번째 깊이 맵 및 RGB 이미지 numpy 배열을 출력합니다.
example_class = list(depth_maps.keys())[0]
example_depth_map = depth_maps[example_class]
example_rgb_image = rgb_images[example_class]

print(f'Numpy array for the first depth map in class {example_class}:')
print(example_depth_map)

print(f'Numpy array for the first RGB image in class {example_class}:')
print(example_rgb_image)

Output hidden; open in https://colab.research.google.com to view.